# Demystifying telemetry.sqlite artifact

When you run logging enabled, a db file will be generated in sqlite format.

See https://microsoft.github.io/autogen/docs/Use-Cases/enhanced_inference/#logging for more info.

In this notebook we will only use two tables, chat_completions and events.

## Prereq

In [ ]:
%pip install jsondiff
%pip install ipywidgets
%pip install markdown

## Load database, and do some data wrangling

In [ ]:
import sqlite3

target_sqlite_db = r"C:\webarena__reddit_TwoAgents\69\0\telemetry.sqlite"

conn = sqlite3.connect(target_sqlite_db)

In [ ]:
import json
from datetime import datetime


# this func gets the image content and returns it - it also removes the from the json to avoid having duplicate data
def get_image_from_event(event):
    image_url = None
    if event["event_name"] == "received_message":
        json_state = event["json_state"]
        for content in json_state["message"]["content"]:
            if content.get("type") == "image_url":
                image_url = content["image_url"]["url"].split(",", 1)[1]
                content["image_url"]["url"] = "<image_placeholder>"
                return image_url
    elif event["event_name"] == "request_to_oai_cc":
        for message in event["request"]["messages"]:
            if isinstance(message["content"], list):
                for subcontent in message["content"]:
                    if subcontent.get("type") == "image_url":
                        image_url = subcontent["image_url"]["url"].split(",", 1)[1]
                        subcontent["image_url"]["url"] = "<image_placeholder>"
                        return image_url
    elif event["event_name"] == "reply_func_executed":
        json_state = event["json_state"]
        for subcontent in json_state["reply"]["content"]:
            if subcontent.get("type") == "image_url":
                image_url = subcontent["image_url"]["url"].split(",", 1)[1]
                subcontent["image_url"]["url"] = "<image_placeholder>"
                return image_url
    else:
        raise Exception(f"image detected in event type {event['event_name']} but not loaded.")

    return image_url


# load table events and do some processing
def db_events_to_list():
    c = conn.cursor()
    c.execute("SELECT * FROM events")
    rows = c.fetchall()
    colnames = c.description

    final = [dict(zip([col[0] for col in colnames], row)) for row in rows]
    for e in final:
        e["image_content"] = "data:image/png;base64" in e["json_state"]
        e["json_state"] = json.loads(e["json_state"])
        if e["image_content"]:
            e["image"] = get_image_from_event(e)
        e["timestamp"] = datetime.strptime(e["timestamp"], "%Y-%m-%d %H:%M:%S.%f")
    return final


# load table chat_completions and do some processing
def db_cc_to_list():
    c = conn.cursor()
    c.execute("SELECT * FROM chat_completions")
    rows = c.fetchall()
    colnames = c.description

    final = [dict(zip([col[0] for col in colnames], row)) for row in rows]
    for e in final:
        e["image_content"] = "data:image/png;base64" in e["request"]
        e["request"] = json.loads(e["request"])
        e["timestamp"] = datetime.strptime(e["start_time"], "%Y-%m-%d %H:%M:%S.%f")
        e["start_time"] = datetime.strptime(e["start_time"], "%Y-%m-%d %H:%M:%S.%f")
        e["end_time"] = datetime.strptime(e["end_time"], "%Y-%m-%d %H:%M:%S.%f")
        e["total_seconds"] = (e["end_time"] - e["start_time"]).seconds
        e["event_name"] = "request_to_oai_cc"
        e["source_name"] = "chat_completions_table"
        if e["image_content"]:
            e["image"] = get_image_from_event(e)
    return final

# Brief explanation

The logs collected by scenario.py in https://github.com/microsoft/autogen/blob/ct_webarena/samples/tools/autogenbench/scenarios/WebArena/Templates/TwoAgents/scenario.py#L50

generate specific events.

In this section we split the list based on these events. We split based on 'source_name' - this way we can keep track of the overall sections inside scenario.py [login phase, setup, actual run, etc]. Then we split this previously split lists by events that have images to simplify the rendering of events with their corresponding last screenshot taken.

In [ ]:
# merge the two lists of the two different tables into a single one
final = db_events_to_list()
print("events total count", len(final))
final2 = db_cc_to_list()
print("chat_completions total count", len(final2))


def merge_lists(list1, list2):
    import heapq

    return list(heapq.merge(list1, list2, key=lambda x: x["timestamp"]))


merged = merge_lists(final, final2)
print("events with images", len([e for e in merged if e["image_content"]]))
print("events with image loaded", len([e for e in merged if e.get("image", None)]))

print("merged total count", len(merged))
filtered_events = [
    event
    for event in merged
    if event["event_name"] != "reply_func_executed"
    or (event["json_state"]["final"] and event["json_state"]["reply"] is not None)
]
filtered_events = [
    event
    for event in filtered_events
    if event["event_name"] != "mws_response"
    and event["event_name"] != "Response"
    and event["event_name"] != "mws_request"
]

print("filtered", len(filtered_events))


def split_on_event(data: list, func, tag: str):
    def group(sequence):
        current_group = {"inner": [], tag: None}
        for item in sequence:
            if func(item):
                if len(current_group["inner"]) > 0 or current_group[tag]:
                    if len(current_group["inner"]) == 0:
                        current_group["inner"].append({})
                    yield current_group
                current_group = {"inner": []}
                current_group[tag] = item
                current_group["inner"] = []
            else:
                current_group["inner"].append(item)

        if len(current_group["inner"]) > 0 or current_group[tag]:
            if len(current_group["inner"]) == 0:
                current_group["inner"].append({})
            yield current_group

    return list(group(data))


merged_and_nested = split_on_event(
    filtered_events, lambda event: event["source_name"] == "scenario.py", tag="scenariopy"
)
print("scenario.py sections", len(merged_and_nested))

for i, section in enumerate(merged_and_nested):
    # section = split_on_event(section["inner"], lambda event: event['event_name'] == "received_message", tag="message")
    section["inner"] = split_on_event(
        section["inner"], lambda event: event.get("image", None) is not None, tag="image_holder"
    )

In [ ]:
print("outer", type(merged_and_nested))
one_run = merged_and_nested[0]
print("one run keys", one_run.keys())
print("inner one run", type(one_run["inner"]))
one_section = one_run["inner"][0]  # one section inside scenario.py
print("one section keys", one_section.keys())
triggered_one_msg = one_section["inner"][0]
print("one single event", triggered_one_msg.keys())

# ipywidgets land - actual UI

In [ ]:
import ipywidgets as widgets
import base64
from IPython.display import display
from markdown import markdown

data = merged_and_nested

# events are stored in lists of lists so we need to keep track of the current event being rendered
# since we have the concept of 'outer events' and 'inner events' we need a type of list of lists
# the outer events represent phases in the driver/orchestrator code and the inner events are
# events caused during that phase
index = [0, 0]  # Using a list to keep track of the index since integers are immutable


def update_image_content(image_widget, event):
    if "image_holder" in event and event["image_holder"] is not None:
        encoded = event["image_holder"]["image"]
        image_widget.value = base64.b64decode(encoded)
        image_widget.width = 800
        image_widget.height = 1000
    else:
        image_widget.value = b""
        image_widget.width = 0
        image_widget.height = 0


def update_content(outer_event_index, inner_event_index):
    outer_event_textarea.value = scenario_py_render(data[outer_event_index]["scenariopy"])
    event_repr, inner_events_repr = event_to_text(data[outer_event_index]["inner"][inner_event_index])
    event_with_image_textarea.value = event_repr
    events_without_image_textarea.value = inner_events_repr

    # load image - the inner_event_index has an image always
    curr_event = data[outer_event_index]["inner"][inner_event_index]

    update_image_content(image_widget, curr_event)


def oai_request_event_md(event):
    output = f"""###  openai api request
- timestamp: **{event["timestamp"]}**
- is_cached: **{event["is_cached"]}**
- cost: **{event["cost"]}**
- total_seconds: **{event["total_seconds"]}**

    """
    for i, message in enumerate(event["request"]["messages"]):
        output += f'message {i}:\n\n {message.get("content", "no_content_message")}\n\n'

    return markdown(output)


def format_event_by_type(event):
    if event is None:
        return "NO INNER EVENT"

    if event["event_name"] == "request_to_oai_cc":
        return oai_request_event_md(event)

    output = f"""event_name: {event["event_name"]}
"""
    if event["event_name"] == "request_to_oai_cc":
        output += f"""timestamp: {event["timestamp"]}
is_cached: {event["is_cached"]}
cost: {event["cost"]}
total_seconds: {event["total_seconds"]}

messages: {json.dumps(event["request"]["messages"], default=lambda o: f"<<non-serializable: {type(o).__qualname__}>>", indent=2)}
"""
    elif event["event_name"] == "reply_func_executed" or event["event_name"] == "received_message":
        output += f"""timestamp: {event["timestamp"]}
inside: {event["source_name"]} type: {event["agent_class_name"]}

json_state: {json.dumps(event["json_state"], default=lambda o: f"<<non-serializable: {type(o).__qualname__}>>", indent=2)}
"""

    return output


def event_with_image_holder_repr(event):
    return format_event_by_type(event)
    # return json.dumps(event, default=lambda o: f"<<non-serializable: {type(o).__qualname__}>>", indent=2)


def event_to_text(event):
    if event:
        if len(event.keys()) == 0:
            return "no event", str(0)
        if len(event["inner"]) == 1 and len(event["inner"][0]) == 0:
            imageless_event_names = "~~~ next event has image too ~~~"
        else:
            imageless_event_names = [e.get("event_name", None) for e in event["inner"]]
        if event["image_holder"]:
            image = event["image_holder"]["image"]
            event["image_holder"]["image"] = "<image placeholder>"
        json_event = event_with_image_holder_repr(event["image_holder"])
        if event["image_holder"]:
            event["image_holder"]["image"] = image
        return json_event, str(imageless_event_names)
    return "no event", str(0)


def transform_scenariopy_event(event):
    assert event["source_name"] == "scenario.py"
    return json.dumps(event, default=lambda o: f"<<non-serializable: {type(o).__qualname__}>>")


def scenario_py_render(event):
    return f"""outermost event: {event["event_name"]}
timestamp: {event["timestamp"]}
inside: {event["source_name"]}

"""
    # return json.dumps(event, default=lambda o: f"<<non-serializable: {type(o).__qualname__}>>")


## render UI

outer_event_textarea = widgets.Textarea(
    value=scenario_py_render(data[index[0]]["scenariopy"]),
    # value=json.dumps(data[index[0]]["scenariopy"], default=lambda o: f"<<non-serializable: {type(o).__qualname__}>>"),
    disabled=False,
    layout={"width": "800px", "height": "60px"},
)

current_event = data[index[0]]["inner"][index[1]]
image_event_repr, inner_events_repr = event_to_text(current_event)

event_with_image_textarea = widgets.HTML(
    value=image_event_repr,
    disabled=False,
    layout={"width": "800px", "height": "200px"},
)

events_without_image_textarea = widgets.Textarea(
    value=inner_events_repr,
    disabled=False,
    layout={"width": "800px", "height": "30px"},
)

image_widget = widgets.Image(
    format="png",
    width=0,
    height=0,
)
update_image_content(image_widget, current_event)
prev_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")


def on_prev_button_clicked(b):
    if index[1] > 0:
        index[1] -= 1
        update_content(outer_event_index=index[0], inner_event_index=index[1])
    elif index[0] > 0:
        index[0] -= 1
        index[1] = len(data[index[0]]["inner"]) - 1
        update_content(outer_event_index=index[0], inner_event_index=index[1])


def on_next_button_clicked(b):
    if index[1] < len(data[index[0]]["inner"]) - 1:
        index[1] += 1
        update_content(outer_event_index=index[0], inner_event_index=index[1])
    elif index[0] < len(data) - 1:
        index[0] += 1
        index[1] = 0
        update_content(outer_event_index=index[0], inner_event_index=index[1])


prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)

# Display widgets
display(
    prev_button,
    next_button,
    outer_event_textarea,
    event_with_image_textarea,
    events_without_image_textarea,
    image_widget,
)

## EXTRA: Diff between states - in this case cookie states

We could also figure out what 'json_state' has changed in between same event types. In this section we try to debug how and when the cookie state changes.

In [ ]:
from jsondiff import diff

cookies_events = [event for event in final if event["event_name"] == "oookies"]
# print when cookie changes
for i in range(1, len(cookies_events)):
    if curr_diff := diff(cookies_events[i - 1]["json_state"], cookies_events[i]["json_state"]):
        print("event number:", cookies_events[i]["id"])
        print(cookies_events[i]["timestamp"])
        print(curr_diff)
        print()

In [ ]:
point_of_interest = 487
context = final[point_of_interest - 35 : point_of_interest]

# drop from context if inside json state url contains 'js' and 'svg'
context = [
    c for c in context if "js" not in c["json_state"].get("url", "") and "svg" not in c["json_state"].get("url", "")
]

for i in range(1, len(context)):
    print("*****", context[i]["id"], context[i]["source_name"], context[i]["event_name"])
    print(context[i])
    if context[i]["event_name"] == "Request":
        print(context[i]["json_state"]["request_content"])
        print(context[i]["json_state"])
    else:
        print(context[i]["json_state"])
    print()